In [1]:
import pandas as pd
import pyodbc 
import datetime as dt

In [2]:
username='maingoc.dinh'
password='Maingoc01'
server = 'GA-9RC9BS2\SQLEXPRESS'
db='TVT_04_2020'
db_Giro='IS_Giro_04_2020'
db_GW='IS_IBanking_04_2020'

m=4
range_day=range(20,27,1) #chọn dài hơn 1 ngày
#range_day_GW=range(1,9,1) #chọn đúng kỳ với cuối tháng?
TVT_day='20_26'


In [3]:
# sql server 2017
conn_str = ('Driver={SQL Server};'
                'Server='+ server+';'
                'Database='+db+';'
                'UID='+username+';'
                'PWD='+password+';')

connection = pyodbc.connect(conn_str)
data=[]
for d in range_day:
    sql_statement = """SELECT * FROM [{2}].[dbo].[TVT_2020-{0:0>2d}-{1:0>2d}-2020-{0:0>2d}-{1:0>2d}]""".format(m,d,db)
    sql_data = pd.read_sql(sql_statement, connection)
    data.append(sql_data)

In [4]:
# data=[]
# for d in range_day:
#     a = pd.read_excel(r'R:\1. DATABASE\1. Bank\2020\Bank 2002\TVT\Transaction detail (TVT) (2020-{0:0>2d}-{1:0>2d}-2020-{0:0>2d}-{1:0>2d}).xlsx'.format(m,d), sheet_name='Sheet1',header=0)
#     data.append(a)

In [5]:
TVT = pd.concat(data)

In [6]:
conn_str_Giro = ('Driver={SQL Server};'
                'Server='+ server+';'
                'Database='+db_Giro+';'
                'UID='+username+';'
                'PWD='+password+';')

connection = pyodbc.connect(conn_str_Giro)

IS_GIRO=[]
for d in range_day:
    sql_statement = """SELECT * FROM [{2}].[dbo].[Giro_2020{0:0>2d}{1:0>2d}_2020{0:0>2d}{1:0>2d}]""".format(m,d,db_Giro)
    sql_data = pd.read_sql(sql_statement, connection)
    IS_GIRO.append(sql_data)

In [7]:
# IS_GIRO=[]
# for d in range_day:
#     a = pd.read_csv(r'R:\1. DATABASE\1. Bank\2020\Bank 2002\Cuong Data\Giro 02_2020\bank_giro_transaction_query_2020{0:0>2d}{1:0>2d}_2020{0:0>2d}{1:0>2d}.csv'.format(m,d),header=0)
#     IS_GIRO.append(a)

In [8]:
IS_GIRO = pd.concat(IS_GIRO)

In [9]:
conn_str_GW = ('Driver={SQL Server};'
                'Server='+ server+';'
                'Database='+db_GW+';'
                'UID='+username+';'
                'PWD='+password+';')

connection_GW = pyodbc.connect(conn_str_GW)

IS_GW=[]
for d in range_day:
    day_IB1 = dt.date(2020,m,d)
    day_IB2 = day_IB1 + dt.timedelta(1)
    m2 = day_IB2.month
    d2 = day_IB2.day 
    
    sql_statement = """SELECT * FROM [{4}].[dbo].[ibanking_2020{0:0>2d}{1:0>2d}_2020{2:0>2d}{3:0>2d}]""".format(m,d,m2,d2,db_GW)
    sql_data = pd.read_sql(sql_statement, connection_GW)
    IS_GW.append(sql_data)
    print(d)

20
21
22
23
24
25
26


In [10]:
# IS_GW=[]
# for d in range_day_GW:
#     day_IB1 = dt.date(2020,m,d)
#     day_IB2 = day_IB1 + dt.timedelta(1)
#     m2 = day_IB2.month
#     d2 = day_IB2.day 
#     a = pd.read_csv(r'R:\1. DATABASE\1. Bank\2020\Bank 2002\Cuong Data\IBanking 02_2020\ibanking_2020{0:0>2d}{1:0>2d}_2020{2:0>2d}{3:0>2d}.csv'.format(m,d,m2,d2,db_GW),header=0,dtype={'System Ref':str})
#     IS_GW.append(a)

In [11]:
IS_GW = pd.concat(IS_GW)

In [12]:
TVT_draft=TVT[(~TVT['TopUp'].str.contains('Cash'))&(~TVT['TopUp'].str.contains('QR'))&(~TVT['TopUp'].str.contains('Credit Card'))&(~TVT['TopUp'].str.contains('Gift'))&(~TVT['TopUp'].str.contains('Virtual Card'))&(~TVT['TopUp'].str.contains('AirPay Counter'))|TVT['PaymentInfo'].isin(['Cash Withdrawal'])]

In [13]:
TVT_info=TVT_draft[(~TVT_draft['TopUp'].str.contains('Shopee Wallet Topup'))|TVT_draft['Promotion'].isin(['Nạp tiền tại Shopee'])]

In [14]:
def Type(TopUp,PaymentInfo):
    if 'Cash Withdrawal' in PaymentInfo:
        i = 'withdraw'
    elif 'Shopee Wallet Topup' in TopUp:
        i='APA Shopee GW'
    elif 'Cash via iBanking' in PaymentInfo:
        i='GW'
    elif 'Cash via Direct Bank Channel Top-up' in PaymentInfo:
        i='API'
    elif 'Cash via Direct Debit' in PaymentInfo or 'Cash via Partner Top-up' in PaymentInfo:
        i='Payment'
    elif 'Cash Transfer' in PaymentInfo or 'Now Transfer' in PaymentInfo or 'Collection QR' in PaymentInfo:
        i='Transfer'
    else:
        i='Shopping'
    return(i)
        

In [15]:
TVT_info['Type']=TVT_info.apply(lambda row:Type(row['TopUp'],row['PaymentInfo']),axis=1)

In [16]:
# TVT_info['OrderID']=TVT_info['OrderID'].astype(int)

In [17]:
# merge TVT_info và Giro để lấy Bank name: và Ref Giro
TVT_info=pd.merge(TVT_info,IS_GIRO,how='left',left_on='OrderID',right_on='Caller Reference')

In [18]:
TVT_info=TVT_info[['OrderID','TopUp','Status','Subtotal','Promotion','Amount_x','CreateTime','PaymentInfo','Product Code','Cash Payable','Channel Payable','UpdateTime','TopUp Ref','Merchant Ref','Topup ValidTime','Buyer Phone','User ID','Payment ValidTime','Ref1','Ref2','Partner Reference IBanking','Type','Bank Name','IS Transaction ID','Bank Transaction ID','Transaction status']]

In [19]:
# merge TVT_info và GW để lấy Bank name và Ref GW
TVT_info=pd.merge(TVT_info,IS_GW,how='left',left_on='OrderID',right_on='Partner Ref')

In [20]:
TVT_info=TVT_info[['OrderID','TopUp','Status_x','Subtotal','Promotion','Amount_x','CreateTime','PaymentInfo','Product Code','Cash Payable','Channel Payable','UpdateTime','TopUp Ref','Merchant Ref','Topup ValidTime','Buyer Phone','User ID','Payment ValidTime','Ref1','Ref2','Partner Reference IBanking','Type','Bank Name','IS Transaction ID','Bank Transaction ID','Transaction status','Gateway','System Ref','Gateway Ref','Status_y']]

In [21]:
TVT_info['Bank_API']=TVT_info['TopUp'].apply(lambda x: x[:len(x)-15:])

In [22]:
IS_GW['Partner Ref']=IS_GW['Partner Ref'].astype(str)

In [23]:
TVT_info=pd.merge(TVT_info,IS_GW,how='left',left_on='Partner Reference IBanking',right_on='Partner Ref')

In [24]:
TVT_info=TVT_info[['OrderID','TopUp','Status_x','Subtotal','Promotion','Amount_x','CreateTime','PaymentInfo','Product Code','Cash Payable','Channel Payable','UpdateTime','TopUp Ref','Merchant Ref','Topup ValidTime','Buyer Phone','User ID','Payment ValidTime','Ref1','Ref2','Partner Reference IBanking','Type','Bank Name','Transaction status','Bank_API','IS Transaction ID','Bank Transaction ID','Gateway_x','System Ref_x','Gateway Ref_x','Status_y','Gateway_y','System Ref_y','Status']]

In [25]:
def Bank(Type,BankName,BankAPI,GatewayX,GatewayY):
    if 'API' in Type:
        r=BankAPI
    elif Type in ['withdraw','Shopping','Payment','Transfer']:
        r=BankName
    elif Type =='GW':
        r=GatewayX
    elif 'APA Shopee GW' in Type:
        r=GatewayY
    else:
        r=''
    return(r)
    

In [26]:
TVT_info['Bank']=TVT_info.apply(lambda row:Bank(row['Type'],row['Bank Name'],row['Bank_API'],row['Gateway_x'],row['Gateway_y']),axis=1)

In [27]:
def RefCheck(TopUpRef,Type,BankName,ISTnxID,BankTnxID,SRefX,SRefY):
    if 'API' in Type:
        m=TopUpRef
    elif Type in ['withdraw','Shopping','Payment','Transfer'] and BankName=='Vietcombank':
    #elif (('withdraw' in Type) or ('Shopping' in Type) or ('Payment' in Type) or ('Transfer' in Type)) and ('Vietcombank' in BankName):
        m=BankTnxID
    elif Type in ['withdraw','Shopping','Payment','Transfer']:
        m=ISTnxID
    elif Type =='GW':
        m=SRefX
    else:
        m=SRefY
    return(m)

In [28]:
TVT_info['Ref_Check']=TVT_info.apply(lambda row:RefCheck(row['TopUp Ref'],row['Type'],row['Bank Name'],row['IS Transaction ID'],row['Bank Transaction ID'],row['System Ref_x'],row['System Ref_y']),axis=1)

In [29]:
def IS_Status(STT1,STT2,STT3):
    if STT1!=None:
        i=STT1
    elif STT2!=None:
        i=STT2
    else:
        i=STT3
    return(i)

In [30]:
TVT_info['IS_Status']=TVT_info.apply(lambda row: IS_Status(row['Transaction status'],row['Status_y'],row['Status']),axis=1)

In [31]:
TVT_info=TVT_info[['OrderID','TopUp','Status_x','Subtotal','Promotion','Amount_x','CreateTime','PaymentInfo','Product Code','Cash Payable','Channel Payable','UpdateTime','TopUp Ref','Merchant Ref','Topup ValidTime','Buyer Phone','User ID','Payment ValidTime','Ref1','Ref2','Partner Reference IBanking','Type','Bank','Ref_Check','IS_Status']]

In [32]:
#Lọc file đối soát
VCB_API= TVT_info[TVT_info['Bank'].isin(['VCB','Vietcombank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['API'])]
VCB_API=VCB_API[['TopUp','Ref_Check','OrderID','Topup ValidTime','Buyer Phone','Amount_x']]

In [33]:
VCB_API.head()

,TopUp,Ref_Check,OrderID,Topup ValidTime,Buyer Phone,Amount_x
25,VCB (Bank Channel),2020042016398515006800000013,212753996,2020-04-20 00:00:15,84896501211,200000
78,VCB (Bank Channel),2020042016400515006800000030,212754105,2020-04-20 00:00:32,84776245546,200000
80,VCB (Bank Channel),2020042070217531806800235929,212754113,2020-04-20 00:00:34,84984649876,80000
132,VCB (Bank Channel),2020042016401515006800000050,212754209,2020-04-20 00:00:52,84799095553,200000
192,VCB (Bank Channel),2020042016402515006800000110,212754321,2020-04-20 00:01:12,84705449529,190000


In [34]:
VCB_Giro=TVT_info[TVT_info['Bank'].isin(['VCB','Vietcombank']) & TVT_info['Type'].isin(['Shopping','Transfer','Payment'])]
VCB_Giro=VCB_Giro[['TopUp Ref','TopUp','OrderID','Topup ValidTime','Buyer Phone','Type','Status_x','Ref_Check','Channel Payable']]

In [35]:
VCB_Giro.head()

,TopUp Ref,TopUp,OrderID,Topup ValidTime,Buyer Phone,Type,Status_x,Ref_Check,Channel Payable
21,vcb_b2w_200420000002,Vietcombank,212753983,2020-04-20 00:00:14,84973649470,Shopping,Success,EW20042022792711,528750
22,vcb_b2w_200420000001,Vietcombank,1029381781,2020-04-20 00:00:13,84977965253,Transfer,Success,EW20042022792710,450000
31,vcb_b2w_200420000005,Vietcombank,212754009,2020-04-20 00:00:18,84938155891,Shopping,Success,EW20042022792723,150000
32,vcb_b2w_200420000006,Vietcombank,212754010,2020-04-20 00:00:18,84702088882,Shopping,Success,EW20042022792725,514591
34,vcb_b2w_200420000007,Vietcombank,212754015,2020-04-20 00:00:20,84962326240,Shopping,Success,EW20042022792727,149000


In [36]:
VCB_GW=TVT_info[TVT_info['Bank'].isin(['VCB','Vietcombank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
VCB_GW=VCB_GW[['Bank','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Amount_x']]

In [37]:
VCB_GW.head()

,Bank,OrderID,Topup ValidTime,Buyer Phone,Ref_Check,Amount_x
1639,VCB,1028669429,2020-04-20 00:19:55,84936468046,200420002100000027,1800000
2694,VCB,1029981512,2020-04-20 00:35:19,84974410266,200420002100000056,1300000
3249,VCB,212760801,2020-04-20 00:43:26,84962641535,200420003100000186,300000
3518,VCB,212761303,2020-04-20 00:49:31,84794756466,200420003100000210,150000
3643,VCB,212761556,2020-04-20 00:53:01,84906319593,200420003100000225,200000


In [38]:
VCB_WD=TVT_info[TVT_info['Bank'].isin(['VCB','Vietcombank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['withdraw'])]
VCB_WD=VCB_WD[['Bank','TopUp Ref','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Subtotal']]

In [39]:
BIDV_API= TVT_info[TVT_info['Bank'].isin(['BIDV']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['API'])]
BIDV_API=BIDV_API[['TopUp','Ref_Check','OrderID','Topup ValidTime','Buyer Phone','Amount_x']]

In [40]:
BIDV_Giro=TVT_info[TVT_info['Bank'].isin(['BIDV']) & TVT_info['Type'].isin(['Shopping','Transfer','Payment'])]
BIDV_Giro=BIDV_Giro[['TopUp Ref','TopUp','OrderID','Topup ValidTime','Buyer Phone','Type','Status_x','Ref_Check','Channel Payable']]

In [41]:
BIDV_WD=TVT_info[TVT_info['Bank'].isin(['BIDV']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['withdraw'])]
BIDV_WD=BIDV_WD[['Bank','TopUp Ref','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Subtotal']]

In [42]:
BIDV_GW=TVT_info[TVT_info['Bank'].isin(['BIDV']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
BIDV_GW=BIDV_GW[['Bank','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Amount_x']]

In [43]:
AGR_API= TVT_info[TVT_info['Bank'].isin(['Agribank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['API'])]
AGR_API=AGR_API[['TopUp','Ref_Check','OrderID','Topup ValidTime','Buyer Phone','Amount_x']]

In [44]:
AGR_Giro=TVT_info[TVT_info['Bank'].isin(['Agribank']) & TVT_info['Type'].isin(['Shopping','Transfer','Payment'])]
AGR_Giro=AGR_Giro[['TopUp Ref','TopUp','OrderID','Topup ValidTime','Buyer Phone','Type','Status_x','Ref_Check','Channel Payable']]

In [45]:
AGR_GW=TVT_info[TVT_info['Bank'].isin(['Agribank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
AGR_GW=AGR_GW[['Bank','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Amount_x']]

In [46]:
AGR_WD=TVT_info[TVT_info['Bank'].isin(['Agribank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['withdraw'])]
AGR_WD=AGR_WD[['Bank','TopUp Ref','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Subtotal']]

In [47]:
VTB_API= TVT_info[TVT_info['Bank'].isin(['Vietinbank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['API'])]
VTB_API=VTB_API[['TopUp','Ref_Check','OrderID','Topup ValidTime','Buyer Phone','Amount_x']]

In [48]:
VTB_Giro=TVT_info[TVT_info['Bank'].isin(['Vietinbank']) & TVT_info['Type'].isin(['Shopping','Transfer','Payment'])]
VTB_Giro=VTB_Giro[['TopUp Ref','TopUp','OrderID','Topup ValidTime','Buyer Phone','Type','Status_x','Ref_Check','Channel Payable']]

In [49]:
VTB_GW=TVT_info[TVT_info['Bank'].isin(['Vietinbank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
VTB_GW=VTB_GW[['Bank','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Amount_x']]

In [50]:
VTB_WD=TVT_info[TVT_info['Bank'].isin(['Vietinbank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['withdraw'])]
VTB_WD=VTB_WD[['Bank','TopUp Ref','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Subtotal']]

In [51]:
API= TVT_info[~TVT_info['Bank'].isin(['BIDV','VCB','Vietcombank','Agribank','Vietinbank','DongABank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['API'])]
API=API[['TopUp','Ref_Check','OrderID','Topup ValidTime','Buyer Phone','Amount_x']]

In [52]:
Giro=TVT_info[~TVT_info['Bank'].isin(['BIDV','VCB','Vietcombank','Agribank','Vietinbank','Sacombank']) & TVT_info['Type'].isin(['Shopping','Transfer','Payment'])]
Giro=Giro[['TopUp Ref','TopUp','OrderID','Topup ValidTime','Buyer Phone','Type','Status_x','Ref_Check','Channel Payable']]

In [53]:
WD=TVT_info[~TVT_info['Bank'].isin(['BIDV','VCB','Vietcombank','Agribank','Vietinbank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['withdraw'])]
WD=WD[['Bank','TopUp Ref','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Subtotal','Merchant Ref']]

In [54]:
GW=TVT_info[~TVT_info['Bank'].isin(['BIDV','VCB','Vietcombank','VNPay','Smart Link','DongABank','Eximbank','Sacombank','ABBank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
GW=GW[['Bank','OrderID','Topup ValidTime','Buyer Phone','Ref_Check','Amount_x']]

In [55]:
Other_DAB_API=TVT_info[TVT_info['Bank'].isin(['DongABank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['API'])]
Other_DAB_API=Other_DAB_API[['Ref_Check','OrderID','CreateTime','Buyer Phone','Amount_x']]

In [56]:
Other_DAB_GW=TVT_info[TVT_info['Bank'].isin(['DongABank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
Other_DAB_GW=Other_DAB_GW[['Bank','OrderID','CreateTime','Buyer Phone','Ref_Check','Subtotal']]

In [57]:
Other_EXB_GW=TVT_info[TVT_info['Bank'].isin(['Eximbank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
Other_EXB_GW=Other_EXB_GW[['Bank','OrderID','CreateTime','Buyer Phone','Ref_Check','Subtotal']]

In [58]:
Other_STB_GW=TVT_info[TVT_info['Bank'].isin(['Sacombank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
Other_STB_GW=Other_STB_GW[['Bank','OrderID','CreateTime','Buyer Phone','Ref_Check','Subtotal']]

In [59]:
Other_STB_Giro=TVT_info[TVT_info['Bank'].isin(['Sacombank']) & TVT_info['Type'].isin(['Shopping','Transfer','Payment'])]
Other_STB_Giro=Other_STB_Giro[['TopUp Ref','TopUp','OrderID','CreateTime','Buyer Phone','Type','Status_x','Ref_Check','Channel Payable']]

In [60]:
Other_ABB_GW=TVT_info[TVT_info['Bank'].isin(['ABBank']) & TVT_info['Status_x'].isin(['Success']) & TVT_info['Type'].isin(['GW','APA Shopee GW'])]
Other_ABB_GW=Other_ABB_GW[['Bank','OrderID','CreateTime','Buyer Phone','Ref_Check','Subtotal']]

In [61]:
WD_autofix=TVT_info[TVT_info['Status_x'].isin(['Autofix Product','Autofix Product Need Staff']) & TVT_info['Type'].isin(['withdraw'])]

In [62]:
TVT_Shopee_refund=TVT_draft[(TVT_draft['TopUp'].isin(['Shopee Wallet Topup'])) & ~ (TVT_draft['Promotion'].str.contains('Nạp tiền tại Shopee',na=False))]

In [63]:
TVT['Destination Account']=TVT['Destination Account'].astype(str)

In [64]:
## Lấy BA
BA_Gift=TVT[TVT['TopUp'].isin(['Gift'])]
#BA_Cash=TVT[TVT['TopUp'].isin(['Cash']) & TVT['Destination Account'].isin(['0'])]
#BA_Cash=TVT[TVT['TopUp'].isin(['Cash']) & TVT['Destination Account'].isin(['0']) | TVT['TopUp'].isin(['Cash']) & TVT['Promotion'].str.contains('#')] 
BA_Cash=TVT[((TVT['TopUp']=='Cash') & (TVT['Destination Account']=='0.0') | (TVT['Destination Account']=='0' )|(TVT['Destination Account']==0)) | ((TVT['TopUp']=='Cash') & (TVT['Promotion'].str.contains('#')))| ((TVT['TopUp']=='Cash') & (TVT['TopUp Ref'].notnull())&(TVT['PaymentInfo']=='Cash Transfer')) ]
BA=pd.concat([BA_Gift,BA_Cash],axis=0)

In [65]:
## Export file
#count_row=VCB_Giro.shape[0]
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\VCB.xlsx'.format(TVT_day,m)) as writer:
    VCB_API.to_excel(writer, sheet_name='VCB_API')
    VCB_Giro.to_excel(writer, sheet_name="VCB_Giro")
#     for i in range(0,count_row,1000000):
#         if i+1000000>count_row:
#             end=count_row
#         else:
#             end=i+1000000
#         sub_df=VCB_Giro.iloc[i:end] 
#         sub_df.to_excel(writer, sheet_name="VCB_Giro +"+str(i))
    VCB_WD.to_excel(writer, sheet_name="VCB_WD")
    VCB_GW.to_excel(writer, sheet_name="VCB_GW")

In [66]:
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\BIDV.xlsx'.format(TVT_day,m)) as writer:
    BIDV_API.to_excel(writer, sheet_name='BIDV_API')
    BIDV_Giro.to_excel(writer, sheet_name="BIDV_Giro")
    BIDV_WD.to_excel(writer, sheet_name="BIDV_WD")
    BIDV_GW.to_excel(writer, sheet_name="BIDV_GW")

In [67]:
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\AGR.xlsx'.format(TVT_day,m)) as writer:
    AGR_API.to_excel(writer, sheet_name='AGR_API')
    AGR_Giro.to_excel(writer, sheet_name="AGR_Giro")
    AGR_WD.to_excel(writer, sheet_name="AGR_WD")
    AGR_GW.to_excel(writer, sheet_name="AGR_GW")

In [68]:
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\VTB.xlsx'.format(TVT_day,m)) as writer:
    VTB_API.to_excel(writer, sheet_name='VTB_API')
    VTB_Giro.to_excel(writer, sheet_name="VTB_Giro")
    VTB_WD.to_excel(writer, sheet_name="VTB_WD")
    VTB_GW.to_excel(writer, sheet_name="VTB_GW")

In [69]:
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\All Bank_exclude_Big4Bank_VNP_SML_Others.xlsx'.format(TVT_day,m)) as writer:
    API.to_excel(writer, sheet_name='API')
    Giro.to_excel(writer, sheet_name="Giro")
    WD.to_excel(writer, sheet_name="WD")
    GW.to_excel(writer, sheet_name="GW")

In [70]:
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\Other.xlsx'.format(TVT_day,m)) as writer:
    Other_DAB_API.to_excel(writer, sheet_name='DAB_API')
    Other_DAB_GW.to_excel(writer, sheet_name="DAB_GW")
    Other_EXB_GW.to_excel(writer, sheet_name="EXB_GW")
    Other_STB_GW.to_excel(writer, sheet_name="STB_GW")
    Other_STB_Giro.to_excel(writer, sheet_name="STB_Giro")
    Other_ABB_GW.to_excel(writer, sheet_name="ABB_GW")

In [71]:
WD_autofix.to_excel(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\WD_autofix.xlsx'.format(TVT_day,m))

In [72]:
count_row=TVT_info.shape[0]
with pd.ExcelWriter(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\TVT_info_{0}.{1:0>2d}.xlsx'.format(TVT_day,m)) as writer:
    for i in range(0,count_row,1000000):
        if i+1000000>count_row:
            end=count_row
        else:
            end=i+1000000
        sub_df=TVT_info.iloc[i:end] 
        sub_df.to_excel(writer, sheet_name="Data +"+str(i))

In [73]:
TVT_Shopee_refund.to_excel(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\TVT {0}\TVT_Shopee_refund_{0}.{1:0>2d}.xlsx'.format(TVT_day,m))

In [74]:
BA.to_excel(r'R:\1. DATABASE\3. Admin toppay\9. TVT Report\{1:0>2d}.2020\BA_{0}T{1:0>2d}.xlsx'.format(TVT_day,m))